In [ ]:
from tensorflow.keras.datasets import cifar10
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import tensorflow as tf
import matplotlib.image as mpimg

In [ ]:
(x_train, y_train), (_, _) = cifar10.load_data()

In [ ]:
# Normalize images to the range [-1, 1]
X_train = (x_train.astype(np.float32) - 127.5) / 127.5

# Convert labels to integer type
y_train = y_train.astype(np.int32)

In [ ]:
X_train = X_train[:30000]
y_train = y_train[:30000]

In [ ]:
X_train.shape

(30000, 32, 32, 3)

In [ ]:
y_train.shape

(30000, 1)

Defining Discriminator

In [ ]:
def build_discriminator():
  in_label = layers.Input(shape=(1,)) #Label input
  li = layers.Embedding(10, 50)(in_label) # 10 is nuber of classes and 50 is the embedding vetor size
  # n_nodes = in_shape[0]*in_shape[1] #32x32
  li = layers.Dense(32*32)(li)
  li = layers.Reshape((32,32,1))(li) #reshape to additional channel 32x32x1 (this is our label input)
  in_image = layers.Input(shape=(32,32,3))
  merge = layers.Concatenate()([in_image, li])
  # Next will be the same classifer code for ncoditional Discriminator
  fe = layers.Conv2D(128, (3,3), strides=2, padding='same')(merge)
  fe = layers.LeakyReLU(alpha=0.2)(fe)
  fe = layers.Conv2D(128, (3,3), strides=2, padding='same')(fe)
  fe = layers.LeakyReLU(alpha=0.2)(fe)
  fe = layers.Flatten()(fe)
  fe = layers.Dropout(0.4)(fe)
  output_layer = layers.Dense(1, activation='sigmoid')(fe)

  model = tf.keras.Model([in_image, in_label], output_layer)
  opt = tf.keras.optimizers.Adam(lr=0.0002)
  model.compile(loss='binary_crossentropy',optimizer=opt, metrics=['accuracy'])
  return model


In [ ]:
def build_generator():
  in_label = layers.Input(shape=(1,))
  li = layers.Embedding(10, 50)(in_label)
  li = layers.Dense(8*8)(li)
  li = layers.Reshape((8,8,1))(li)
  in_noise = layers.Input(shape=(100,))
  gen = layers.Dense(8*8*128)(in_noise) # after upscaling we will get 32x32x3
  gen = layers.LeakyReLU(alpha=0.2)(gen)
  gen = layers.Reshape((8,8,128))(gen)
  merge = layers.Concatenate()([gen,li])

  gen = layers.Conv2DTranspose(128, (4,4), strides=2, padding='same')(merge)
  gen = layers.LeakyReLU(alpha=0.2)(gen)
  gen = layers.Conv2DTranspose(128, (4,4), strides=2, padding='same')(gen)
  gen = layers.LeakyReLU(alpha=0.2)(gen)

  output_layer = layers.Conv2D(3, (8,8), activation='tanh', padding='same')(gen) # Output shape will be 32x32x3
  model = tf.keras.Model([in_noise, in_label], output_layer)
  return model

In [ ]:
disc = build_discriminator()
gen = build_generator()
disc.trainable = False

z = layers.Input(shape=(100,))
label = layers.Input(shape=(1,), dtype = 'int32')
img = gen([z, label])
output = disc([img, label])

model = tf.keras.Model([z, label], output)
opt = tf.keras.optimizers.Adam(lr=0.0002)
model.compile(loss='binary_crossentropy', optimizer=opt)


In [ ]:
def train():
  epochs=10000
  batch=50
  noise_dim=100
  for i in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], batch)
    real_images, real_labels = X_train[idx], y_train[idx]
    y_real = np.ones((batch,1))
    real_loss, _ = disc.train_on_batch([real_images, real_labels], y_real)

    noise = tf.random.normal([batch, noise_dim])
    noise_labels = np.random.randint(0,10,batch)
    fake_images = gen.predict([noise,noise_labels])
    display_images(fake_images,0)
    y_fake = np.zeros((batch,1))

    fake_loss, _ = disc.train_on_batch([fake_images, noise_labels], y_fake)
    y_gan = tf.ones(batch)

    g_loss = model.train_on_batch([noise, noise_labels], y_gan)

    print(f'epoch: {i+1}')
    print(f'loss: {g_loss}')

  model.save('cifar_conditional_generator,h5')


In [ ]:
def normalize_image(image):
    return (image + 1) / 2



def display_images(image_list, start_index):
    fig, axes = plt.subplots(4, 4, figsize=(8, 8))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    for i, ax in enumerate(axes.flat):
        if start_index + i < len(image_list):
            img = normalize_image(image_list[start_index + i])
            ax.imshow(img)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.show()
    plt.close()

In [ ]:
train()

**GPT**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers.legacy import Adam


In [ ]:
(x_train, y_train), (_, _) = cifar10.load_data()

# Normalize images to the range [-1, 1]
x_train = (x_train.astype(np.float32) - 127.5) / 127.5

# Convert labels to integer type
y_train = y_train.astype(np.int32)


In [ ]:
def build_generator():
    noise_shape = (100,)
    label_shape = (1,)

    noise = layers.Input(shape=noise_shape)
    label = layers.Input(shape=label_shape, dtype='int32')

    label_embedding = layers.Embedding(10, 50, input_length=1)(label)
    label_embedding = layers.Flatten()(label_embedding)

    model_input = layers.Concatenate()([noise, label_embedding])

    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_dim=150))  # 100 noise + 50 for embedding
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(1024))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.Dense(32 * 32 * 3, activation='tanh'))
    model.add(layers.Reshape((32, 32, 3)))

    img = model(model_input)

    return tf.keras.Model([noise, label], img)

def build_discriminator():
    img_shape = (32, 32, 3)
    label_shape = (1,)

    img = layers.Input(shape=img_shape)
    label = layers.Input(shape=label_shape, dtype='int32')

    label_embedding = layers.Embedding(10, 32*32*3)(label)
    label_embedding = layers.Flatten()(label_embedding)
    label_embedding = layers.Reshape((32, 32, 3))(label_embedding)

    model_input = layers.Concatenate(axis=-1)([img, label_embedding])

    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, kernel_size=3, strides=2, input_shape=(32, 32, 6), padding="same"))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(layers.ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(layers.BatchNormalization(momentum=0.8))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    validity = model(model_input)

    return tf.keras.Model([img, label], validity)


In [ ]:
optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()
z = layers.Input(shape=(100,))
label = layers.Input(shape=(1,), dtype='int32')
img = generator([z, label])

discriminator.trainable = False
valid = discriminator([img, label])

combined = tf.keras.Model([z, label], valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


In [ ]:
import os
os.makedirs("images", exist_ok=True)

def sample_images(epoch):
    r, c = 4, 4  # 4x4 grid for 16 images
    noise = np.random.normal(0, 1, (r * c, 100))
    sampled_labels = np.tile(np.arange(0, 10).reshape(-1, 1), (r * c // 10 + 1, 1))[:r * c]
    gen_imgs = generator.predict([noise, sampled_labels])

    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c, figsize=(10, 10))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,:])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/cgan_%d.png" % epoch)
    plt.close()

epochs = 10000
batch_size = 64
sample_interval = 200

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs, labels = x_train[idx], y_train[idx]

    noise = np.random.normal(0, 1, (batch_size, 100))
    gen_imgs = generator.predict([noise, labels])

    d_loss_real = discriminator.train_on_batch([imgs, labels], valid)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

    g_loss = combined.train_on_batch([noise, sampled_labels], valid)

    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if epoch % sample_interval == 0:
        sample_images(epoch)


Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 4ms/step
3236 [D loss: 0.000001, acc.: 100.00%] [G loss: 6.547801]
2/2 [==============================] - 0s 3ms/step
3237 [D loss: 0.000001, acc.: 100.00%] [G loss: 6.754858]
2/2 [==============================] - 0s 5ms/step
3238 [D loss: 0.000000, acc.: 100.00%] [G loss: 7.077684]
2/2 [==============================] - 0s 4ms/step
3239 [D loss: 0.000001, acc.: 100.00%] [G loss: 7.004251]
2/2 [==============================] - 0s 4ms/step
3240 [D loss: 0.000000, acc.: 100.00%] [G loss: 6.452400]
2/2 [==============================] - 0s 6ms/step
3241 [D loss: 0.000000, acc.: 100.00%] [G loss: 6.728974]
2/2 [==============================] - 0s 4ms/step
3242 [D loss: 0.000001, acc.: 100.00%] [G loss: 6.969250]
2/2 [==============================] - 0s 4ms/step
3243 [D loss: 0.000000, acc.: 100.00%] [G loss: 6.694065]
2/2 [==============================] - 0s 11ms/step
3244 [D loss: 0.000000, a

KeyboardInterrupt: 